In [36]:
import numpy as np
import os
import time
import pickle
import cv2
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from keras import backend as K
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model, load_model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Input
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.optimizers import Adam, RMSprop, SGD
from keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from skimage.transform import resize
from sklearn.svm import SVC

In [37]:
labels = []
feature_list = []
feature_dict=pickle.load(open('features_till_flatten.pkl','rb'))
names = [0,1]
for cat,features  in feature_dict.items():
    labels.extend([int(cat)] * len(features))
    feature_list.append(features)
feature_list = np.concatenate(feature_list)

In [38]:
feature_list1=[]
for i in range(0,len(feature_list)):
    feature_list1.append(feature_list[i].ravel())

In [39]:
feature_list1[0].shape

(25088,)

In [40]:
scaler = StandardScaler().fit(feature_list1)
rescaledX = scaler.transform(feature_list1)
num_folds = 10
seed = 7
kfold = KFold(n_splits=num_folds, random_state=seed)
model = SVC()
results = cross_val_score(model, rescaledX, labels)
print ("Accuracy: {} ({})".format(results.mean()*100.0, results.std()*100.0))

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change fro

Accuracy: 85.99222567828014 (1.401219809014135)


In [ ]:
scoring = 'accuracy'

c_values = [0.1, 0.3, 0.5, 0.7, 0.9, 1.0, 1.3, 1.5, 1.7, 2.0]
kernel_values = [ 'linear' , 'poly' , 'rbf' , 'sigmoid' ]
param_grid = dict(C=c_values, kernel=kernel_values)

model = SVC()

kfold = KFold(n_splits=num_folds, random_state=seed)

grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)

grid_result = grid.fit(rescaledX, labels)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [46]:
tuned_model = SVC(C=0.1, kernel='rbf')

In [47]:
tuned_model.fit(rescaledX, labels)

/usr/local/lib/python3.5/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [48]:
labels2 = []
feature_list = []
feature_dict=pickle.load(open('features_till_flatten_validation.pkl','rb'))
names = [0,1]
for cat,features  in feature_dict.items():
    labels2.extend([int(cat)] * len(features))
    feature_list.append(features)
feature_list = np.concatenate(feature_list)

In [49]:
feature_list2=[]
for i in range(0,len(feature_list)):
    feature_list2.append(feature_list[i].ravel())

In [50]:
rescaledX_validation = scaler.transform(feature_list2)

result = tuned_model.score(rescaledX_validation, labels2)
print('Accuracy: %.3f%%' % (result * 100.0))

Accuracy: 76.923%
